Goal: Load 3-5 images for the 10 most-likely species near a given location.


In [ ]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2

In [2]:
# Load previously-saved weighted species checklists for locations of interest
from pathlib import Path

import yaml

yaml_file_stem = "probabilities-1767313880"
yaml_path = Path.cwd().parent / f"data/{yaml_file_stem}.yaml"

with yaml_path.open() as infile:
    species_prob_data = yaml.safe_load(infile)
species_prob_data

{'Providence, Rhode Island, United States': {'blujay': 0.011199778850727912,
  'amerob': 0.011089723926958343,
  'sonspa': 0.011040696534817058,
  'norcar': 0.010942771366663608,
  'amegfi': 0.010826700864686517,
  'moudov': 0.010822196414735925,
  'bkcchi': 0.010698583591829547,
  'carwre': 0.010693119999528608,
  'amecro': 0.010643939363553845,
  'dowwoo': 0.010544182277998701,
  'houspa': 0.010462133604078279,
  'comgra': 0.010386441818823536,
  'tuftit': 0.010354045242755934,
  'cangoo': 0.010350446938337261,
  'eursta': 0.01021835535481509,
  'norfli': 0.010169681819430057,
  'rewbla': 0.010105421490942991,
  'houfin': 0.010089747341899878,
  'grycat': 0.009975007771320992,
  'mallar3': 0.009814923638324523,
  'rebwoo': 0.009802861678323857,
  'whbnut': 0.009707051732368542,
  'daejun': 0.009656500999111715,
  'rethaw': 0.009645064575295096,
  'whtspa': 0.009426399519243673,
  'normoc': 0.00942426337520259,
  'chiswi': 0.00935838363652682,
  'grbher3': 0.0091558844818198,
  'chisp

In [3]:
from birding.ebird_api import get_ebird_api_key, retrieve_taxonomy_entries
from birding.sqlite_cache import init_db

species_codes = ["blujay", "amerob", "sonspa", "norcar", "amegfi"]

ebird_api_key = get_ebird_api_key()

init_db()
example_taxonomy = retrieve_taxonomy_entries(ebird_api_key, species_codes)
example_taxonomy

[{'sciName': 'Cyanocitta cristata',
  'comName': 'Blue Jay',
  'speciesCode': 'blujay',
  'category': 'species',
  'taxonOrder': 21294.0,
  'bandingCodes': ['BLJA'],
  'comNameCodes': [],
  'sciNameCodes': ['CYCR'],
  'order': 'Passeriformes',
  'familyCode': 'corvid1',
  'familyComName': 'Crows, Jays, and Magpies',
  'familySciName': 'Corvidae'},
 {'sciName': 'Turdus migratorius',
  'comName': 'American Robin',
  'speciesCode': 'amerob',
  'category': 'species',
  'taxonOrder': 28633.0,
  'bandingCodes': ['AMRO'],
  'comNameCodes': ['ALRO'],
  'sciNameCodes': ['TUMI'],
  'order': 'Passeriformes',
  'familyCode': 'turdid1',
  'familyComName': 'Thrushes and Allies',
  'familySciName': 'Turdidae'},
 {'sciName': 'Melospiza melodia',
  'comName': 'Song Sparrow',
  'speciesCode': 'sonspa',
  'category': 'species',
  'taxonOrder': 33277.0,
  'bandingCodes': ['SOSP'],
  'comNameCodes': [],
  'sciNameCodes': ['MEME'],
  'order': 'Passeriformes',
  'familyCode': 'passer3',
  'familyComName': 'N

In [4]:
locations_of_interest = ["Providence, Rhode Island, United States"]

In [5]:
from birding.primitives import TaxonomyEntry

taxonomy_entries: dict[str, TaxonomyEntry] = {}
for loc in locations_of_interest:
    loc_species_probs = species_prob_data[loc]
    loc_species_ids = list(loc_species_probs.keys())
    entries_data = retrieve_taxonomy_entries(ebird_api_key, species=loc_species_ids)
    for entry_data in entries_data:
        entry = TaxonomyEntry.from_json(entry_data)
        taxonomy_entries[entry.ebird_species_code] = entry

taxonomy_entries

Calling eBird API for the taxonomy entry of 16 species...
Unable to retrieve eBird taxonomy entry for species: x00004
Unable to retrieve eBird taxonomy entry for species: cxbgol1
Unable to retrieve eBird taxonomy entry for species: x00759
Unable to retrieve eBird taxonomy entry for species: x00730
Unable to retrieve eBird taxonomy entry for species: tdxsca1
Unable to retrieve eBird taxonomy entry for species: brewar
Unable to retrieve eBird taxonomy entry for species: x00724
Unable to retrieve eBird taxonomy entry for species: x00628
Unable to retrieve eBird taxonomy entry for species: lawwar
Unable to retrieve eBird taxonomy entry for species: x00776
Unable to retrieve eBird taxonomy entry for species: x00421
Unable to retrieve eBird taxonomy entry for species: x00613
Unable to retrieve eBird taxonomy entry for species: x01131
Unable to retrieve eBird taxonomy entry for species: x00686
Unable to retrieve eBird taxonomy entry for species: x00611
Unable to retrieve eBird taxonomy entry 

{'blujay': TaxonomyEntry(common_name='Blue Jay', scientific_name='Cyanocitta cristata', ebird_species_code='blujay', order='Passeriformes', family_common_name='Crows, Jays, and Magpies', family_scientific_name='Corvidae'),
 'amerob': TaxonomyEntry(common_name='American Robin', scientific_name='Turdus migratorius', ebird_species_code='amerob', order='Passeriformes', family_common_name='Thrushes and Allies', family_scientific_name='Turdidae'),
 'sonspa': TaxonomyEntry(common_name='Song Sparrow', scientific_name='Melospiza melodia', ebird_species_code='sonspa', order='Passeriformes', family_common_name='New World Sparrows', family_scientific_name='Passerellidae'),
 'norcar': TaxonomyEntry(common_name='Northern Cardinal', scientific_name='Cardinalis cardinalis', ebird_species_code='norcar', order='Passeriformes', family_common_name='Cardinals and Allies', family_scientific_name='Cardinalidae'),
 'amegfi': TaxonomyEntry(common_name='American Goldfinch', scientific_name='Spinus tristis', ebi

How can I access images for a specific species? Is there any way to automatically quality control the images?


In [23]:
from pyinaturalist import Observations, pprint

from birding.inaturalist import retrieve_photo_observations
from birding.sqlite_cache import init_db

init_db()

n_test_species = 1
day_of_month = 3

taxonomy_entries_list = list(taxonomy_entries.values())
for i in range(n_test_species):
    taxon_name = taxonomy_entries_list[-i].scientific_name

    result = retrieve_photo_observations(taxon_name, day_of_month)
    observations = Observations.from_json(result)
    pprint(observations)

iNaturalist photo observations for 'Cyanocitta cristata' were already cached.
                                                                                
  ID          Taxon ID   Taxon         Observed on   User          Location     
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  329332615   8229       Cyanocitta    Dec 03,       andybridges   Montgomery   
                         cristata      2025                        County, MO,  
                         (Blue Jay)                                USA          
  286655618   8229       Cyanocitta    Jun 03,       paul_solarz   E Berkshire  
                         cristata      2025                        Ln, Mount    
                         (Blue Jay)                                Prospect,    
                                                                   IL, US       
  278527947   8229       Cyanocitta    May 03,       sdoss56       Old          
                         crista

In [30]:
# Retrieve photo observations for the top 10 species near the location(s) of interest
top_10_taxonomy_entries = {}
for loc in locations_of_interest:
    species_probs = species_prob_data[loc]
    sorted_species = sorted([(s, p) for s, p in species_probs.items()], key=lambda x: -x[1])
    top_10_species = [s for s, _ in sorted_species[:10]]
    top_10_entries_data = retrieve_taxonomy_entries(ebird_api_key, top_10_species)

    top_10_taxonomy_entries[loc] = [TaxonomyEntry.from_json(data) for data in top_10_entries_data]

top_10_taxonomy_entries

{'Providence, Rhode Island, United States': [TaxonomyEntry(common_name='Blue Jay', scientific_name='Cyanocitta cristata', ebird_species_code='blujay', order='Passeriformes', family_common_name='Crows, Jays, and Magpies', family_scientific_name='Corvidae'),
  TaxonomyEntry(common_name='American Robin', scientific_name='Turdus migratorius', ebird_species_code='amerob', order='Passeriformes', family_common_name='Thrushes and Allies', family_scientific_name='Turdidae'),
  TaxonomyEntry(common_name='Song Sparrow', scientific_name='Melospiza melodia', ebird_species_code='sonspa', order='Passeriformes', family_common_name='New World Sparrows', family_scientific_name='Passerellidae'),
  TaxonomyEntry(common_name='Northern Cardinal', scientific_name='Cardinalis cardinalis', ebird_species_code='norcar', order='Passeriformes', family_common_name='Cardinals and Allies', family_scientific_name='Cardinalidae'),
  TaxonomyEntry(common_name='American Goldfinch', scientific_name='Spinus tristis', ebird

In [36]:
from collections import defaultdict

day_of_month = 5
observations_per_species = defaultdict(list)
for loc, top_10 in top_10_taxonomy_entries.items():
    print(f"Finding iNaturalist observations for top 10 species at '{loc}'...")
    for taxon_entry in top_10:
        taxon_name = taxon_entry.scientific_name
        result = retrieve_photo_observations(taxon_name, day_of_month)
        observations = Observations.from_json(result)
        observations_per_species[taxon_name].append(observations)
        pprint(observations)

Finding iNaturalist observations for top 10 species at 'Providence, Rhode Island, United States'...
iNaturalist photo observations for 'Cyanocitta cristata' were already cached.
                                                                                
  ID          Taxon ID   Taxon         Observed on   User          Location     
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  329665811   8229       Cyanocitta    Dec 05,       mhalsted      Flinton, ON  
                         cristata      2025                        K0H 1P0,     
                         (Blue Jay)                                Canada       
  329665806   8229       Cyanocitta    Dec 05,       mhalsted      Flinton, ON  
                         cristata      2025                        K0H 1P0,     
                         (Blue Jay)                                Canada       
  304005670   8229       Cyanocitta    Aug 05,       kgibs1        Water St,    
            

In [ ]:
# Check photo size in the downloaded photo observations
photos_per_species = defaultdict(list)
"""Map from species names to lists of photos downloaded from iNaturalist."""

min_image_dim = 1024

for species_name, observations_list in observations_per_species.items():
    for observations in observations_list:
        for obs in observations.data:
            for photo in obs.photos:
                if min(photo.original_dimensions) >= min_image_dim:
                    photos_per_species[species_name].append(photo)

for species, photos in photos_per_species.items():
    print(f"Downloaded {len(photos)} photos for species '{species}'.")

Downloaded 59 photos for species 'Cyanocitta cristata'.
Downloaded 32 photos for species 'Turdus migratorius'.
Downloaded 46 photos for species 'Melospiza melodia'.
Downloaded 41 photos for species 'Cardinalis cardinalis'.
Downloaded 35 photos for species 'Spinus tristis'.
Downloaded 39 photos for species 'Zenaida macroura'.
Downloaded 39 photos for species 'Poecile atricapillus'.
Downloaded 54 photos for species 'Thryothorus ludovicianus'.
Downloaded 48 photos for species 'Corvus brachyrhynchos'.
Downloaded 91 photos for species 'Dryobates pubescens'.


In [40]:
import ipyplot

labeled_thumbnails = [(p.medium_url, s) for s, ps in photos_per_species.items() for p in ps]
images = [url for url, _ in labeled_thumbnails]
labels = [label for _, label in labeled_thumbnails]
ipyplot.plot_class_tabs(images, labels, max_imgs_per_tab=15)